In [136]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('train.csv')
df.info()
df.isna().sum()
df.duplicated().sum()
df.drop_duplicates()
label =  LabelEncoder()
df['model'] = label.fit_transform(df['model'])
df['motor_type'] = label.fit_transform(df['motor_type'])
# df['wheel'] = df['wheel'].map({'left': 1})
df['wheel'] = label.fit_transform(df['wheel'])
df['color'] = label.fit_transform(df['color'])
df['status'] = label.fit_transform(df['status'])
df['type'] = label.fit_transform(df['type'])
def convert_running(value):
    if 'km' in value:
        kilometers = int(value.split()[0])  
        miles = kilometers * 0.621371        
        return f"{miles:.2f} miles"          
    elif 'miles' in value:
        return value  
    else:
        return value 
df['running'] = df['running'].apply(convert_running)
df['running'] = df['running'].str.replace('miles', '')




df['running'] = df['running'].str.strip()
df['running'] = pd.to_numeric(df['running'], errors='coerce')
df['running'] = df['running'].fillna(0).astype('int64')


df.corr()['price']



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   object 
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null   object 
 3   running       1642 non-null   object 
 4   wheel         1642 non-null   object 
 5   color         1642 non-null   object 
 6   type          1642 non-null   object 
 7   status        1642 non-null   object 
 8   motor_volume  1642 non-null   float64
 9   price         1642 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 128.4+ KB


model           0.173485
year            0.638699
motor_type      0.238964
running        -0.513134
wheel                NaN
color          -0.036846
type            0.037789
status         -0.282471
motor_volume    0.000885
price           1.000000
Name: price, dtype: float64

In [138]:
x_train = df.drop(['price'], axis = 1)
y_train = df['price']
x_train.shape, y_train.shape

((1642, 9), (1642,))

In [140]:
from sklearn.preprocessing import StandardScaler
scalar =  StandardScaler()
feature_standardise = scalar.fit_transform(x_train)
feature_standardise

array([[ 1.52639053,  1.09243073,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ],
       [ 0.05769246, -0.12234706,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ],
       [-0.67665657,  0.48504183,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ],
       ...,
       [ 0.79204149,  0.48504183,  0.26699337, ...,  1.08693988,
         0.97583205, -0.1384163 ],
       [ 0.79204149,  0.63688906,  0.26699337, ...,  1.08693988,
        -0.45353122, -0.1384163 ],
       [ 1.52639053,  1.09243073,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ]])

In [144]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

# Update parameter grid to include 'loss' and 'max_features'
param_distributions = {
    
    'n_estimators': [100, 300, 500, 700, 1000],  # Higher values to boost performance
    'learning_rate': [0.001, 0.01, 0.05, 0.1],  # Smaller learning rates combined with more estimators
    'max_depth': [3, 4, 5],  # Fewer deep trees to prevent overfitting
    'min_samples_split': [5, 10, 15],  # More samples required to split a node
    'min_samples_leaf': [2, 4, 6],  # More samples required at leaf nodes
    'subsample': [0.6, 0.7, 0.8, 0.9],  # Introduce more randomness in the subsample
    'max_features': ['sqrt', 'log2'],  # Limit the number of features used per tree
    'loss': ['huber'],  # Huber loss function to reduce sensitivity to outliers
    'alpha': [0.75, 0.85, 0.9]  # Fine-tuning alpha, only relevant for huber loss
}

# Initialize the GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42)

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=gbr,
    param_distributions=param_distributions,
    n_iter=100,
    scoring='neg_mean_absolute_error',  # Still optimizing for MAE
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Fit the model on standardized features
random_search.fit(feature_standardise, y_train)

# Get the best parameters
best_params = random_search.best_params_
print(f'Best parameters: {best_params}')

# Get the best model
best_model = random_search.best_estimator_
print(best_model)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters: {'subsample': 0.9, 'n_estimators': 300, 'min_samples_split': 15, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 4, 'loss': 'huber', 'learning_rate': 0.05, 'alpha': 0.75}
GradientBoostingRegressor(alpha=0.75, learning_rate=0.05, loss='huber',
                          max_depth=4, max_features='log2', min_samples_leaf=2,
                          min_samples_split=15, n_estimators=300,
                          random_state=42, subsample=0.9)


In [146]:
gbr = GradientBoostingRegressor(alpha=0.75, learning_rate=0.05, loss='huber',
                          max_depth=4, max_features='log2', min_samples_leaf=2,
                          min_samples_split=15, n_estimators=300,
                          random_state=42, subsample=0.9)
gbr.fit(feature_standardise, y_train)

GradientBoostingRegressor(alpha=0.75, learning_rate=0.05, loss='huber',
                          max_depth=4, max_features='log2', min_samples_leaf=2,
                          min_samples_split=15, n_estimators=300,
                          random_state=42, subsample=0.9)

In [79]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.metrics import mean_absolute_error
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# import numpy as np

# # Assuming you have your features and target variable defined as feature_standardise and y_train
# # Scale your features


# # Define the parameter grid
# param_distributions = {
#     'n_estimators': [50, 100, 200, 300],
#     'learning_rate': [0.01, 0.1, 0.2, 0.3],
#     'max_depth': [3, 4, 5, 6, 7, 8],
#     'min_samples_split': [2, 5, 10, 15],
#     'min_samples_leaf': [1, 2, 4, 7],
#     'subsample': [0.5, 0.75, 1.0],
#     'max_features': ['auto', 'sqrt', 'log2', None],
#     'loss': ['ls', 'lad', 'huber'],  # Include different loss functions
# }

# # Initialize the GradientBoostingRegressor
# gbr = GradientBoostingRegressor(random_state=42)

# # Perform RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     estimator=gbr,
#     param_distributions=param_distributions,
#     n_iter=100,
#     scoring='neg_mean_absolute_error',  # Optimizing for MAE
#     cv=5,
#     verbose=1,
#     random_state=42,
#     n_jobs=-1
# )

# # Fit the model on standardized features
# random_search.fit(feature_standardise, y_train)

# # Get the best parameters
# best_params = random_search.best_params_
# print(f'Best parameters: {best_params}')

# # Get the best model
# best_model = random_search.best_estimator_
# print(best_model)

# # If you want to consider the 'alpha' parameter based on the best 'loss'
# if best_params['loss'] in ['huber', 'quantile']:
#     best_alpha = best_params.get('alpha', None)
#     print(f'Best alpha (if applicable): {best_alpha}')

# # Optionally evaluate the model on test data if available
# # y_pred = best_model.predict(X_test_standardised)
# # test_mae = mean_absolute_error(y_test, y_pred)
# # print(f'Test Mean Absolute Error: {test_mae}')


Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
360 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\admin\anaconda3\Lib\site-package

Best parameters: {'subsample': 0.75, 'n_estimators': 50, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 7, 'loss': 'huber', 'learning_rate': 0.1}
GradientBoostingRegressor(loss='huber', max_depth=7, max_features='log2',
                          min_samples_leaf=2, min_samples_split=5,
                          n_estimators=50, random_state=42, subsample=0.75)
Best alpha (if applicable): None


In [81]:
gbr = GradientBoostingRegressor(loss='huber', max_depth=7, max_features='log2',
                          min_samples_leaf=2, min_samples_split=5,
                          n_estimators=50, random_state=42, subsample=0.75)
gbr.fit(feature_standardise, y_train)

GradientBoostingRegressor(loss='huber', max_depth=7, max_features='log2',
                          min_samples_leaf=2, min_samples_split=5,
                          n_estimators=50, random_state=42, subsample=0.75)

In [157]:
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score

y_pred = gbr.predict(feature_standardise)
mse = mean_squared_error(y_train, y_pred)
print(f'Mean Squared Error with Gradient Boosting: {mse:.2f}')
mae = mean_absolute_error(y_train, y_pred)
print(mae)
r2 = r2_score(y_train, y_pred)
print(r2)
# Mean Squared Error with Gradient Boosting: 7959291.73
# 1664.5559299893227
# 0.8453450271621639
# Mean Squared Error with Gradient Boosting: 9786569.17
# 1644.2758685809188
# 0.8098396640173866
# Mean Squared Error with Gradient Boosting: 11025533.24
# 1743.8036736129643
# 0.7857656683361012

# from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score

# y_pred = gbr.predict(feature_standardise)
# mse = mean_squared_error(y_train, y_pred)
# print(f'Mean Squared Error with Gradient Boosting: {mse:.2f}')
# mae = mean_absolute_error(y_train, y_pred)
# print(mae)
# r2 = r2_score(y_train, y_pred)
# print(r2)
# Mean Squared Error with Gradient Boosting: 7959291.73
# 1664.5559299893227
# 0.8453450271621639
# Mean Squared Error with Gradient Boosting: 9786569.17
# 1644.2758685809188
# 0.8098396640173866
# Mean Squared Error with Gradient Boosting: 11025533.24
# 1743.8036736129643
# 0.7857656683361012
# Mean Squared Error with Gradient Boosting: 9503979.20
# 1618.4430885088304
# 0.8153305977970433
# Mean Squared Error with Gradient Boosting: 9417658.89
# 1614.256197660024
# 0.8170078658748969
# ean Squared Error with Gradient Boosting: 9179245.84
# 1596.0584620193215
# 0.8216404091534659
# Mean Squared Error with Gradient Boosting: 9179245.84
# 1596.0584620193215
# 0.8216404091534659
# Mean Squared Error with Gradient Boosting: 9518499.92
# 1618.4517975963006
# 0.8150484494763944
# Mean Squared Error with Gradient Boosting: 9565921.43
# 1624.5620892743634
# 0.8141270141475943

# from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score

# y_pred = gbr.predict(feature_standardise)
# mse = mean_squared_error(y_train, y_pred)
# print(f'Mean Squared Error with Gradient Boosting: {mse:.2f}')
# mae = mean_absolute_error(y_train, y_pred)
# print(mae)
# r2 = r2_score(y_train, y_pred)
# print(r2)
# Mean Squared Error with Gradient Boosting: 7959291.73
# 1664.5559299893227
# 0.8453450271621639
# Mean Squared Error with Gradient Boosting: 9786569.17
# 1644.2758685809188
# 0.8098396640173866
# Mean Squared Error with Gradient Boosting: 11025533.24
# 1743.8036736129643
# 0.7857656683361012

# from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score

# y_pred = gbr.predict(feature_standardise)
# mse = mean_squared_error(y_train, y_pred)
# print(f'Mean Squared Error with Gradient Boosting: {mse:.2f}')
# mae = mean_absolute_error(y_train, y_pred)
# print(mae)
# r2 = r2_score(y_train, y_pred)
# print(r2)
# Mean Squared Error with Gradient Boosting: 7959291.73
# 1664.5559299893227
# 0.8453450271621639
# Mean Squared Error with Gradient Boosting: 9786569.17
# 1644.2758685809188
# 0.8098396640173866
# Mean Squared Error with Gradient Boosting: 11025533.24
# 1743.8036736129643
# 0.7857656683361012
# Mean Squared Error with Gradient Boosting: 9503979.20
# 1618.4430885088304
# 0.8153305977970433
# Mean Squared Error with Gradient Boosting: 9417658.89
# 1614.256197660024
# 0.8170078658748969
# ean Squared Error with Gradient Boosting: 9179245.84
# 1596.0584620193215
# 0.8216404091534659
# Mean Squared Error with Gradient Boosting: 9179245.84
# 1596.0584620193215
# 0.8216404091534659--------------
# Mean Squared Error with Gradient Boosting: 9518499.92
# 1618.4517975963006
# 0.8150484494763944
# Mean Squared Error with Gradient Boosting: 9565921.43
# 1624.5620892743634
# 0.8141270141475943

# Mean Squared Error with Gradient Boosting: 7879109.25
# 1418.467824277582
# 0.8469030324481694

Mean Squared Error with Gradient Boosting: 9348302.91
1601.9441250576729
0.8183555043000252


In [159]:
test = pd.read_csv('test.csv')
from sklearn.preprocessing import LabelEncoder
label =  LabelEncoder()


test['model'] = label.fit_transform(test['model'])
test['motor_type'] = label.fit_transform(test['motor_type'])
test['wheel'] = label.fit_transform(test['wheel'])
# test['wheel'] = test['wheel'].map({'left': 1})
test['color'] = label.fit_transform(test['color'])
test['status'] = label.fit_transform(test['status'])
test['type'] = label.fit_transform(test['type'])
def convert_runnings(value):
    if 'km' in value:
        kilometers = int(value.split()[0])  
        miles = kilometers * 0.621371        
        return f"{miles:.2f} miles"          
    elif 'miles' in value:
        return value  
    else:
        return value 

test['running'] = test['running'].apply(convert_runnings)
test['running'] = test['running'].str.replace('miles', '')
x_test = test

In [161]:
x_test = x_test.drop(['Id'], axis = 1)

In [163]:
from sklearn.preprocessing import StandardScaler
scalar =  StandardScaler()
feature_standard = scalar.fit_transform(x_test)
feature_standard

array([[-0.62247891,  0.77117487,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538],
       [ 0.8692927 ,  0.31182288,  0.1137996 , ...,  1.45476435,
        -0.4470452 , -0.10974538],
       [-1.36836471,  0.9242922 ,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538],
       ...,
       [ 0.1234069 , -0.45376377,  0.1137996 , ..., -0.05892716,
         1.04673998, -0.10974538],
       [-0.62247891,  0.77117487,  0.1137996 , ..., -0.05892716,
         1.04673998, -0.10974538],
       [-1.36836471,  0.31182288,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538]])

In [170]:
y_predict = gbr.predict(feature_standard)
y_predict
df_train = pd.DataFrame(y_predict, columns=['price'])
df_train

,price
0,16730.682477
1,16222.357255
2,24288.571021
3,14361.481529
4,5067.316233
...,...
406,26652.810411
407,13633.569409
408,12267.407531
409,15430.529362


In [174]:
combined_df = pd.concat([test['Id'], df_train], axis=1)

combined_df

,Id,price
0,0,16730.682477
1,1,16222.357255
2,2,24288.571021
3,3,14361.481529
4,4,5067.316233
...,...,...
406,406,26652.810411
407,407,13633.569409
408,408,12267.407531
409,409,15430.529362


In [176]:
combined_df.to_csv('Car_com.csv', index=False)